In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import numpy as np

from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import cross_val_score
from keras.wrappers.scikit_learn import KerasRegressor

from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [ ]:
directory_crime = "Europe Regional 2023 Datasets/Traffic, Investigations _ Other/crimes.csv"
df_crime = pd.read_csv(directory_crime)

cleandf_crime = pd.DataFrame()

year = []
month = []
day = []
time = []
for i in df_crime['dispatch_date_time']:
    i_datetime = i.split(' ')
    i_date = i_datetime[0].split('-')
    
    year.append(int(i_date[0]))
    month.append(int(i_date[1]))
    day.append(int(i_date[2]))
    
    i_time = float(i_datetime[1][:2])+float(i_datetime[1][3:5])/60
    time.append(i_time)

cleandf_crime['year'] = year
cleandf_crime['month'] = month
cleandf_crime['day'] = day
cleandf_crime['time'] = time

assault, burglary, rape, robbery, theft, other_crime = [], [], [], [], [], []

for i in df_crime['text_general_code']:
    c = [0,0,0,0,0,0]
    if 'Assault' in i:
        c[0] = 1
    elif 'Burglary' in i:
        c[1] = 1
    elif 'Rape' in i:
        c[2] = 1
    elif 'Robbery' in i:
        c[3] = 1
    elif 'Theft' in i:
        c[4] = 1
    else:
        c[5] = 1
    assault.append(c[0])
    burglary.append(c[1])
    rape.append(c[2])
    robbery.append(c[3])
    theft.append(c[4])
    other_crime.append(c[5])

cleandf_crime['assault'] = assault
cleandf_crime['burglary'] = burglary
cleandf_crime['rape'] = rape
cleandf_crime['robbery'] = robbery
cleandf_crime['theft'] = theft
cleandf_crime['other_crime'] = other_crime  

cleandf_crime['latitude'] = df_crime['lat']
cleandf_crime['longitude'] = df_crime['lng']

crime_x = cleandf_crime.iloc[:,:4]
crime_y = cleandf_crime.iloc[:,4:10]
crime_xstd = MinMaxScaler().fit_transform(crime_x.values)

In [ ]:
model = Sequential()
model.add(Dense(32, activation='tanh', input_dim=4))
model.add(Dense(16, activation='tanh'))
model.add(Dense(6, activation='softmax'))
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.fit(x=crime_xstd,y=crime_y,batch_size=256,validation_split=0.1,epochs=50,verbose=1)